In [ ]:
!virtualenv -p python3 venv
!chmod +x venv/bin/activate
!source ./venv/bin/activate

Install Python Librairie

In [ ]:
!pip uninstall -r requirements.txt -y
# !pip install -r requirements.txt


In [41]:
!pip uninstall -y llama-index
!pip uninstall -y langchain
!pip uninstall -y gradio
!pip uninstall -y python-dotenv
!pip uninstall -y ipython

Found existing installation: llama-index 0.8.40
Uninstalling llama-index-0.8.40:
  Successfully uninstalled llama-index-0.8.40
Found existing installation: langchain 0.0.309
Uninstalling langchain-0.0.309:
  Successfully uninstalled langchain-0.0.309
Found existing installation: gradio 3.47.1
Uninstalling gradio-3.47.1:
  Successfully uninstalled gradio-3.47.1
Found existing installation: python-dotenv 1.0.0
Uninstalling python-dotenv-1.0.0:
  Successfully uninstalled python-dotenv-1.0.0
Found existing installation: ipython 8.16.1
Uninstalling ipython-8.16.1:
  Successfully uninstalled ipython-8.16.1


In [42]:
!pip install llama-index
# !pip install langchain
!pip install langchain==0.0.148
!pip install gradio
!pip install python-dotenv
!pip install ipython

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/bc/01/0d3d74f373ac37eba8198aca82098437406b70be45990695c62f84156046/llama_index-0.8.40-py3-none-any.whl.metadata
  Using cached llama_index-0.8.40-py3-none-any.whl.metadata (5.0 kB)
  Obtaining dependency information for langchain>=0.0.303 from https://files.pythonhosted.org/packages/7e/01/3236ef65067dee4172a7735fc34c43a80d61b3a8632fb5a8f0db37f345db/langchain-0.0.309-py3-none-any.whl.metadata
  Using cached langchain-0.0.309-py3-none-any.whl.metadata (15 kB)
Using cached llama_index-0.8.40-py3-none-any.whl (866 kB)
Using cached langchain-0.0.309-py3-none-any.whl (1.8 MB)
  Using cached langchain-0.0.148-py3-none-any.whl (636 kB)
  Obtaining dependency information for SQLAlchemy<2,>=1 from https://files.pythonhosted.org/packages/77/68/4ce3f0677a4c5f51a91624a7c41921ea39aac1e39502d252ff339ec6cd3b/SQLAlchemy-1.4.49-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux

In [44]:
from langchain import OpenAI
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,readers, GPTSimpleVectorIndex, PromptHelper
# from llama_index import LLMPredictor, ServiceContext
import sys
#from google.colab import drive
import os

ImportError: cannot import name 'GPTSimpleVectorIndex' from 'llama_index' (/home/nkaurelien/Workspaces/Ysoconseils/playground/venv/lib/python3.10/site-packages/llama_index/__init__.py)

Create a .env file

Example:


OPENAI_API_KEY=sk-
OPENAI_API_MODEL=gpt-3.5-turbo
OPENAI_API_MODEL=text-davinci-003
OPENAI_TEMPATURE=0.5
KNOWLEDGE_BASE_PATH=/home/nkaurelien/Workspaces/ysoconseils/faqs_silae/public_html
INPUT_INDEX=index.json


In [ ]:
# os.environ["OPENAI_API_KEY"] = ''
from dotenv import load_dotenv

load_dotenv()

In [ ]:
input_index = os.getenv('INPUT_INDEX', default='index.json') 

In [ ]:
def construct_index(directory_path):
  # set maximum input size
  max_input_size = 4096
  # set number of output tokens
  num_outputs = 256
  # set maximum chunk overlap
  max_chunk_overlap = 20
  # set chunk size limit
  chunk_size_limit = 600

  prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

  # define LLM
  llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-002", max_tokens=num_outputs))
  
  documents = SimpleDirectoryReader(directory_path).load_data()
  
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
  index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
  
  index.save_to_disk('index.json')
  
  return index

In [ ]:
def ask_bot():
  index = GPTSimpleVectorIndex.load_from_disk(input_index)
  while True:
    query = input('What do you want to ask the bot?   \n')
    response = index.query(query, response_mode="compact")
    print ("\nBot says: \n\n" + response.response + "\n\n\n")

In [ ]:
import gradio as gr

def chatbot(input_text):
    index = GPTSimpleVectorIndex.load_from_disk(input_index)
    # response = index.query(input_text, response_mode="compact")
    response = index.query(input_text)
    return response.response


def ask_bot_web_interface():

    iface = gr.Interface(fn=chatbot,
                     inputs=gr.components.Textbox(lines=4, label="Entrer votre texte"),
                     outputs="text",
                     title="Custom-trained AI Chatbot")
    iface.launch(share=True)

In [ ]:
docs_path = os.getenv("KNOWLEDGE_BASE_PATH", default="/content/")
index = construct_index(docs_path)

In [ ]:

ask_bot()

In [ ]:
ask_bot_web_interface()